In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from keras.layers import BatchNormalization, Lambda, Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from keras.layers.merge import Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action = 'ignore', category= FutureWarning)
%matplotlib inline


In [2]:
def mycolournet(num_classes):
    input_image = Input(shape=(56,56,3))
    layer1 = MaxPooling2D(pool_size=(1,1),strides=(1,1),input_shape=(56,56,3))(input_image)
    
    conv1 = Convolution2D(filters=32,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(layer1)
    conv1 = BatchNormalization()(conv1)
    conv1 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(conv1)
    
    conv2 = Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(conv1)
    conv2 = BatchNormalization()(conv2)
    conv2 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(conv2)
    
    conv3 =  Convolution2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu',padding='same')(conv2)
    conv3 = BatchNormalization()(conv3)
    conv3 = MaxPooling2D(pool_size=(3,3),strides=(2,2))(conv3)
    
    concatoutput = Flatten()(conv2)
    
    commonFC_1 = Dense(units=20, activation='relu'  )(concatoutput)
    commonFC_2 = Dense(units=50, activation = 'relu')(commonFC_1)
    commonFC_3 = Dense(units=30, activation= 'relu')(commonFC_2)
    output = Dense(units=num_classes,activation='softmax' )(commonFC_3)
    
    model = Model(inputs=input_image,outputs=output)
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    #sgd = SGD(lr=0.05, momentum=0.9, decay=0.0005, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [58]:
model = mycolournet(2)

In [59]:
train_path = 'data/colors/train'
valid_path = 'data/colors/valid'
test_path = 'data/colors/test'

In [60]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(56,56), classes=['black','grey'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(56,56), classes=[  'black','grey'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(56,56), classes=[ 'black','grey' ], batch_size=10, shuffle=False)

Found 163 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [61]:
model.fit(x=train_batches,
    steps_per_epoch=len(train_batches),
    validation_data=valid_batches,
    validation_steps=len(valid_batches),
    epochs=100,
    verbose=2)          

Epoch 1/100
17/17 - 4s - loss: 0.4696 - accuracy: 0.8221
Epoch 2/100
17/17 - 1s - loss: 0.1263 - accuracy: 0.9693
Epoch 3/100
17/17 - 1s - loss: 0.0753 - accuracy: 0.9632
Epoch 4/100
17/17 - 1s - loss: 0.1547 - accuracy: 0.9448
Epoch 5/100
17/17 - 1s - loss: 0.0767 - accuracy: 0.9816
Epoch 6/100
17/17 - 1s - loss: 0.0265 - accuracy: 0.9939
Epoch 7/100
17/17 - 1s - loss: 0.0112 - accuracy: 1.0000
Epoch 8/100
17/17 - 1s - loss: 0.0211 - accuracy: 0.9877
Epoch 9/100
17/17 - 1s - loss: 0.0042 - accuracy: 1.0000
Epoch 10/100
17/17 - 1s - loss: 0.0105 - accuracy: 0.9939
Epoch 11/100
17/17 - 1s - loss: 0.0042 - accuracy: 1.0000
Epoch 12/100
17/17 - 2s - loss: 0.0021 - accuracy: 1.0000
Epoch 13/100
17/17 - 1s - loss: 0.0019 - accuracy: 1.0000
Epoch 14/100
17/17 - 1s - loss: 0.0013 - accuracy: 1.0000
Epoch 15/100
17/17 - 1s - loss: 0.0054 - accuracy: 1.0000
Epoch 16/100
17/17 - 2s - loss: 0.0019 - accuracy: 1.0000
Epoch 17/100
17/17 - 2s - loss: 0.0014 - accuracy: 1.0000
Epoch 18/100
17/17 - 1s

KeyboardInterrupt: 

In [27]:
model = mycolournet(4)

In [28]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(56,56), classes=['blue', 'red', 'yellow', 'brown'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(56,56), classes=['blue', 'red', 'yellow', 'brown'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(56,56), classes=['blue', 'red', 'yellow', 'brown'], batch_size=10, shuffle=False)

Found 590 images belonging to 4 classes.
Found 12 images belonging to 4 classes.
Found 2 images belonging to 4 classes.


In [29]:
model.fit(x=train_batches,
    steps_per_epoch=len(train_batches),
    validation_data=valid_batches,
    validation_steps=len(valid_batches),
    epochs=20,
    verbose=2)   

Epoch 1/20
59/59 - 30s - loss: 1.2102 - accuracy: 0.4017 - val_loss: 1.2395 - val_accuracy: 0.7500
Epoch 2/20
59/59 - 28s - loss: 1.0035 - accuracy: 0.5102 - val_loss: 1.1750 - val_accuracy: 0.5000
Epoch 3/20
59/59 - 30s - loss: 0.9887 - accuracy: 0.5576 - val_loss: 1.0463 - val_accuracy: 0.4167
Epoch 4/20
59/59 - 30s - loss: 0.9200 - accuracy: 0.6000 - val_loss: 0.9132 - val_accuracy: 0.9167
Epoch 5/20
59/59 - 29s - loss: 0.7953 - accuracy: 0.6729 - val_loss: 0.6043 - val_accuracy: 1.0000
Epoch 6/20
59/59 - 29s - loss: 0.7210 - accuracy: 0.7017 - val_loss: 0.7984 - val_accuracy: 0.5833
Epoch 7/20
59/59 - 29s - loss: 0.7141 - accuracy: 0.7085 - val_loss: 0.4658 - val_accuracy: 1.0000
Epoch 8/20
59/59 - 29s - loss: 0.6371 - accuracy: 0.7695 - val_loss: 0.5155 - val_accuracy: 1.0000
Epoch 9/20
59/59 - 29s - loss: 0.6685 - accuracy: 0.7424 - val_loss: 0.5696 - val_accuracy: 0.9167
Epoch 10/20
59/59 - 29s - loss: 0.6739 - accuracy: 0.7271 - val_loss: 0.5291 - val_accuracy: 1.0000
Epoch 11/

In [39]:
model = mycolournet()

In [40]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(56,56), classes=['blue', 'red', 'yellow', 'brown'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(56,56), classes=['blue', 'red', 'yellow', 'brown'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(56,56), classes=['blue', 'red', 'yellow', 'brown'], batch_size=10, shuffle=False)

Found 590 images belonging to 4 classes.
Found 12 images belonging to 4 classes.
Found 2 images belonging to 4 classes.


In [41]:
model.fit(x=train_batches,
    steps_per_epoch=len(train_batches),
    validation_data=valid_batches,
    validation_steps=len(valid_batches),
    epochs=20,
    verbose=2)  

Epoch 1/20
59/59 - 31s - loss: 1.2680 - accuracy: 0.3898 - val_loss: 1.3134 - val_accuracy: 0.4167
Epoch 2/20
59/59 - 29s - loss: 0.9520 - accuracy: 0.6153 - val_loss: 1.1495 - val_accuracy: 0.5000
Epoch 3/20
59/59 - 29s - loss: 0.7269 - accuracy: 0.6780 - val_loss: 0.5975 - val_accuracy: 0.5000
Epoch 4/20
59/59 - 30s - loss: 0.7055 - accuracy: 0.6542 - val_loss: 0.4983 - val_accuracy: 0.5000
Epoch 5/20
59/59 - 31s - loss: 0.6654 - accuracy: 0.6915 - val_loss: 0.4917 - val_accuracy: 0.5000
Epoch 6/20
59/59 - 31s - loss: 0.5701 - accuracy: 0.7508 - val_loss: 0.4008 - val_accuracy: 0.7500
Epoch 7/20
59/59 - 29s - loss: 0.5662 - accuracy: 0.7458 - val_loss: 0.2479 - val_accuracy: 0.9167
Epoch 8/20
59/59 - 29s - loss: 0.5356 - accuracy: 0.7949 - val_loss: 0.2942 - val_accuracy: 0.9167
Epoch 9/20
59/59 - 30s - loss: 0.5558 - accuracy: 0.7831 - val_loss: 0.1450 - val_accuracy: 1.0000
Epoch 10/20
59/59 - 29s - loss: 0.4721 - accuracy: 0.8203 - val_loss: 0.1634 - val_accuracy: 1.0000
Epoch 11/

In [6]:
model = mycolournet(9)

In [7]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(56,56), classes=['blue', 'red', 'yellow', 'brown', 'green', 'grey','orange','purple','pink'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(56,56), classes=['blue', 'red', 'yellow', 'brown', 'green', 'grey','orange','purple','pink'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(56,56), classes=['blue', 'red', 'yellow', 'brown', 'green', 'grey','orange','purple','pink'], batch_size=10, shuffle=False)

Found 1282 images belonging to 9 classes.
Found 83 images belonging to 9 classes.
Found 2 images belonging to 9 classes.


In [8]:
model.fit(x=train_batches,
    steps_per_epoch=len(train_batches),
    validation_data=valid_batches,
    validation_steps=len(valid_batches),
    epochs=50,
    verbose=2) 

Epoch 1/50


C:\Users\Debjyoty\anaconda3\lib\site-packages\PIL\Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


129/129 - 82s - loss: 2.0879 - accuracy: 0.2340 - val_loss: 1.8139 - val_accuracy: 0.3373
Epoch 2/50
129/129 - 60s - loss: 1.7396 - accuracy: 0.3495 - val_loss: 1.5663 - val_accuracy: 0.3976
Epoch 3/50
129/129 - 60s - loss: 1.6766 - accuracy: 0.3619 - val_loss: 1.5967 - val_accuracy: 0.2892
Epoch 4/50
129/129 - 63s - loss: 1.5886 - accuracy: 0.3783 - val_loss: 1.4633 - val_accuracy: 0.3494
Epoch 5/50
129/129 - 63s - loss: 1.5561 - accuracy: 0.3768 - val_loss: 1.4249 - val_accuracy: 0.3133
Epoch 6/50
129/129 - 63s - loss: 1.4965 - accuracy: 0.4041 - val_loss: 1.4228 - val_accuracy: 0.3855
Epoch 7/50
129/129 - 63s - loss: 1.4747 - accuracy: 0.3861 - val_loss: 1.4991 - val_accuracy: 0.3253
Epoch 8/50
129/129 - 63s - loss: 1.3837 - accuracy: 0.4056 - val_loss: 1.4790 - val_accuracy: 0.3133
Epoch 9/50
129/129 - 63s - loss: 1.3806 - accuracy: 0.4337 - val_loss: 1.3952 - val_accuracy: 0.3253
Epoch 10/50
129/129 - 63s - loss: 1.3736 - accuracy: 0.4306 - val_loss: 1.3859 - val_accuracy: 0.3373


KeyboardInterrupt: 

In [6]:
model = mycolournet(9)

In [7]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(56,56), classes=['blue', 'red', 'yellow', 'brown', 'green', 'grey','orange','purple','pink'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(56,56), classes=['blue', 'red', 'yellow', 'brown', 'green', 'grey','orange','purple','pink'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(56,56), classes=['blue', 'red', 'yellow', 'brown', 'green', 'grey','orange','purple','pink'], batch_size=10, shuffle=False)

Found 1098 images belonging to 9 classes.
Found 83 images belonging to 9 classes.
Found 2 images belonging to 9 classes.


In [8]:
model.fit(x=train_batches,
    steps_per_epoch=len(train_batches),
    validation_data=valid_batches,
    validation_steps=len(valid_batches),
    epochs=50,
    verbose=2) 

Epoch 1/50


C:\Users\Debjyoty\anaconda3\lib\site-packages\PIL\Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


110/110 - 54s - loss: 2.1859 - accuracy: 0.1512 - val_loss: 2.1224 - val_accuracy: 0.2169
Epoch 2/50
110/110 - 48s - loss: 2.0853 - accuracy: 0.2268 - val_loss: 1.9823 - val_accuracy: 0.2169
Epoch 3/50
110/110 - 48s - loss: 2.0163 - accuracy: 0.2295 - val_loss: 1.8917 - val_accuracy: 0.1807
Epoch 4/50
110/110 - 49s - loss: 1.8644 - accuracy: 0.2158 - val_loss: 1.7842 - val_accuracy: 0.2410
Epoch 5/50
110/110 - 48s - loss: 1.7579 - accuracy: 0.2851 - val_loss: 1.8275 - val_accuracy: 0.2892
Epoch 6/50
110/110 - 48s - loss: 1.7240 - accuracy: 0.3297 - val_loss: 1.7687 - val_accuracy: 0.2530
Epoch 7/50
110/110 - 48s - loss: 1.6994 - accuracy: 0.3424 - val_loss: 1.7461 - val_accuracy: 0.2892
Epoch 8/50
110/110 - 48s - loss: 1.6468 - accuracy: 0.3552 - val_loss: 1.7259 - val_accuracy: 0.2771
Epoch 9/50
110/110 - 49s - loss: 1.6331 - accuracy: 0.3770 - val_loss: 1.7260 - val_accuracy: 0.3253
Epoch 10/50
110/110 - 49s - loss: 1.6197 - accuracy: 0.3597 - val_loss: 1.7178 - val_accuracy: 0.3253


In [4]:
model = mycolournet(10)

In [5]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(56,56), classes=['blue', 'red', 'yellow', 'brown', 'green', 'grey','orange','purple','pink', 'white'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(56,56), classes=['blue', 'red', 'yellow', 'brown', 'green', 'grey','orange','purple','pink', 'white'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(56,56), classes=['blue', 'red', 'yellow', 'brown', 'green', 'grey','orange','purple','pink', 'white'], batch_size=10, shuffle=False)

Found 1238 images belonging to 10 classes.
Found 83 images belonging to 10 classes.
Found 2 images belonging to 10 classes.


In [6]:
model.fit(x=train_batches,
    steps_per_epoch=len(train_batches),
    validation_data=valid_batches,
    validation_steps=len(valid_batches),
    epochs=100,
    verbose=2) 

Epoch 1/100


C:\Users\Debjyoty\anaconda3\lib\site-packages\PIL\Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


124/124 - 62s - loss: 2.2517 - accuracy: 0.1858 - val_loss: 2.0363 - val_accuracy: 0.2651
Epoch 2/100
124/124 - 60s - loss: 2.0715 - accuracy: 0.2553 - val_loss: 2.0008 - val_accuracy: 0.2048
Epoch 3/100
124/124 - 61s - loss: 1.9976 - accuracy: 0.2851 - val_loss: 2.0046 - val_accuracy: 0.2289
Epoch 4/100
124/124 - 60s - loss: 1.9112 - accuracy: 0.3037 - val_loss: 1.9434 - val_accuracy: 0.2530
Epoch 5/100
124/124 - 61s - loss: 1.8622 - accuracy: 0.3288 - val_loss: 2.0301 - val_accuracy: 0.2289
Epoch 6/100
124/124 - 61s - loss: 1.8238 - accuracy: 0.3441 - val_loss: 1.9561 - val_accuracy: 0.2651
Epoch 7/100
124/124 - 61s - loss: 1.7706 - accuracy: 0.3813 - val_loss: 1.8268 - val_accuracy: 0.2651
Epoch 8/100
124/124 - 61s - loss: 1.6974 - accuracy: 0.3918 - val_loss: 2.0165 - val_accuracy: 0.2289
Epoch 9/100
124/124 - 61s - loss: 1.6588 - accuracy: 0.3998 - val_loss: 1.8378 - val_accuracy: 0.2410
Epoch 10/100
124/124 - 62s - loss: 1.6071 - accuracy: 0.4176 - val_loss: 1.7497 - val_accuracy

Epoch 81/100
124/124 - 61s - loss: 0.5525 - accuracy: 0.7738 - val_loss: 1.8961 - val_accuracy: 0.4337
Epoch 82/100
124/124 - 60s - loss: 0.5463 - accuracy: 0.7868 - val_loss: 1.7747 - val_accuracy: 0.4699
Epoch 83/100
124/124 - 8222s - loss: 0.5084 - accuracy: 0.7973 - val_loss: 1.9146 - val_accuracy: 0.4458
Epoch 84/100
124/124 - 67s - loss: 0.5262 - accuracy: 0.7940 - val_loss: 1.7539 - val_accuracy: 0.4458
Epoch 85/100
124/124 - 59s - loss: 0.5129 - accuracy: 0.7900 - val_loss: 1.7017 - val_accuracy: 0.4337
Epoch 86/100
124/124 - 32323s - loss: 0.5611 - accuracy: 0.7868 - val_loss: 2.0402 - val_accuracy: 0.4458
Epoch 87/100
124/124 - 60s - loss: 0.5236 - accuracy: 0.7908 - val_loss: 1.7533 - val_accuracy: 0.5181
Epoch 88/100
124/124 - 79s - loss: 0.5259 - accuracy: 0.7981 - val_loss: 1.9837 - val_accuracy: 0.4458
Epoch 89/100
124/124 - 62s - loss: 0.5044 - accuracy: 0.8069 - val_loss: 1.8154 - val_accuracy: 0.3735
Epoch 90/100
124/124 - 63s - loss: 0.4952 - accuracy: 0.8037 - val_l

In [10]:
model = mycolournet(10)

In [11]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(56,56), classes=['blue', 'red', 'yellow', 'brown', 'green', 'grey','orange','purple','pink', 'white'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(56,56), classes=['blue', 'red', 'yellow', 'brown', 'green', 'grey','orange','purple','pink', 'white'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(56,56), classes=['blue', 'red', 'yellow', 'brown', 'green', 'grey','orange','purple','pink', 'white'], batch_size=10, shuffle=False)

Found 1422 images belonging to 10 classes.
Found 83 images belonging to 10 classes.
Found 2 images belonging to 10 classes.


In [12]:
model.fit(x=train_batches,
    steps_per_epoch=len(train_batches),
    validation_data=valid_batches,
    validation_steps=len(valid_batches),
    epochs=150,
    verbose=2) 

Epoch 1/150


C:\Users\Debjyoty\anaconda3\lib\site-packages\PIL\Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


143/143 - 85s - loss: 1.3925 - accuracy: 0.5260 - val_loss: 0.8760 - val_accuracy: 0.6988
Epoch 2/150
143/143 - 12s - loss: 0.6710 - accuracy: 0.7855 - val_loss: 0.8220 - val_accuracy: 0.7470
Epoch 3/150
143/143 - 12s - loss: 0.5803 - accuracy: 0.8129 - val_loss: 0.3855 - val_accuracy: 0.8675
Epoch 4/150
143/143 - 11s - loss: 0.4700 - accuracy: 0.8530 - val_loss: 0.3891 - val_accuracy: 0.8554
Epoch 5/150
143/143 - 12s - loss: 0.4073 - accuracy: 0.8734 - val_loss: 0.3037 - val_accuracy: 0.9277
Epoch 6/150
143/143 - 12s - loss: 0.3488 - accuracy: 0.8734 - val_loss: 0.2168 - val_accuracy: 0.9036
Epoch 7/150
143/143 - 12s - loss: 0.2688 - accuracy: 0.9142 - val_loss: 0.3395 - val_accuracy: 0.8916
Epoch 8/150
143/143 - 12s - loss: 0.2614 - accuracy: 0.9086 - val_loss: 0.2963 - val_accuracy: 0.9157
Epoch 9/150
143/143 - 12s - loss: 0.2082 - accuracy: 0.9381 - val_loss: 0.3895 - val_accuracy: 0.8916
Epoch 10/150
143/143 - 12s - loss: 0.2651 - accuracy: 0.9177 - val_loss: 0.3949 - val_accuracy

KeyboardInterrupt: 

In [7]:
model.save('models/mycolourmodel.h5')

In [8]:
from tensorflow.keras.models import load_model
new_model = load_model('models/mycolourmodel.h5')

In [9]:
new_model.fit(x=train_batches,
    steps_per_epoch=len(train_batches),
    validation_data=valid_batches,
    validation_steps=len(valid_batches),
    epochs=10,
    verbose=2)
   

ValueError: Failed to find data adapter that can handle input: <class 'NoneType'>, <class 'NoneType'>

In [62]:
model.save('models/blackgreymodel.h5')


In [14]:
x=model.predict(test_batches)

In [15]:
x

array([[9.9990141e-01, 6.4550411e-05, 3.3962293e-05]], dtype=float32)

In [16]:
np.argmax(x)

0